In [1]:
class Quantity: #1
    def __init__(self, storage_name):
        self.storage_name = storage_name #2
    
    def __set__(self, instance, value): #3
        if value > 0:
            instance.__dict__[self.storage_name] = value #4
        else:
            raise ValueError('value must be > 0')
            

class LineItem:
    weight = Quantity('weight') #5
    price = Quantity('price') #6
    
    def __init__(self, description, weight, price): #7
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [2]:
truffle = LineItem('White truffle', 100, 0)

ValueError: value must be > 0

In [18]:
class Quantity:
    __counter = 0 #1
    
    def __init__(self):
        cls = self.__class__ #2
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index) #3
        cls.__counter += 1 #4
        
    def __get__(self, instance, owner): #5
        return getattr(instance, self.storage_name) #6
    
    def __set__(self, instance, value): #6
        if value > 0:
            setattr(instance, self.storage_name, value) #7
        else:
            raise ValueError('value must be > 0')
            
class LineItem:
    weight = Quantity() #9
    price = Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [19]:
coconuts = LineItem('Brazilian coconut', 20, 17.95)

In [20]:
coconuts.weight, coconuts.price

(20, 17.95)

In [21]:
getattr(coconuts, '_Quantity#0')

20

In [24]:
LineItem.weight

In [23]:
class Quantity:
    __counter = 0 #1
    
    def __init__(self):
        cls = self.__class__ #2
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index) #3
        cls.__counter += 1 #4
        
    def __get__(self, instance, owner): #5
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name) #6
    
    def __set__(self, instance, value): #6
        if value > 0:
            setattr(instance, self.storage_name, value) #7
        else:
            raise ValueError('value must be > 0')
            
class LineItem:
    weight = Quantity() #9
    price = Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [ ]:
import model_v4c as model


class LineItem:
    weight = model.Quantity()
    price = model.Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [25]:
def quantity(): #1
    try:
        quantity.counter += 1 #2
    except AttributeError:
        quantity.counter = 0 #3
    storage_name = '_{}:{}'.format('quntity', quantity.counter) #4
    
    def qty_getter(instance): #5
        return getattr(instance, storage_name)
    
    def qty_setter(instance, value):
        if value > 0:
            setattr(instance, storage_name, value)
        else:
            raise ValueError('value must be > 0')
    
    return property(qty_getter, qty_setter)

In [ ]:
import abc

class AutoStorage: #1
    __counter = 0
    
    
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)
        
    def __set__(self, instance, value):
        setattr(instance, self.storage_name, value) #2
        

class Validated(abc.ABC, AutoStorage): #3
    def __set__(self, instance, value):
        value = self.validate(instance, value) #4
        super().__set__(instance, value) #5
        
    @abc.abstractmethod
    def validate(self, instance, value): #6
        """return validated value or raise ValueError"""
        

class Quantity(Validated): #7
    """a number greater than zero"""
    def validate(self, instance, value):
        if value <= 0:
            raise ValueError('value must be > 0')
        return value
    

class NonBlank(Validated):
    """a string with at least one ono-space character"""
    def validate(self, instance, value):
        value = value.strip()
        if len(value) == 0:
            raise ValueError('value cannot be empty or blank')
        return value #8
    
class LineItem:
    description = NonBlank()
    weight = Quantity()
    price = Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [1]:
def cls_name(obj_or_cls):
    cls = type(obj_or_cls)
    if cls is type:
        cls = obj_or_cls
    return cls.__name__.split('.')[-1]

def display(obj):
    cls = type(obj)
    if cls is type:
        return '<class {}>'.format(obj.__name__)
    elif cls in [type(None), int]:
        return repr(obj)
    else:
        return '<{} object>'.format(cls_name(obj))
    
def print_args(name, *args):
    pseudo_args = ', '.join(display(x) for x in args)
    print('-> {}.__{}__({})'.format(cls_name(args[0]), name, pseudo_args))
    
class Overriding: #1
    """也称数据描述符或强制描述符"""
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner) #2
        
    def __set__(self, instance, value):
        print_args('set', self, instance, value)
        
class OverridingNoGet: #3
    """没有``__get__``方法的覆盖性描述符"""
    def __set__(self, instance, value):
        print_args('set', self, instance, value)
        
        
class NonOverriding: #4
    """也称非数据描述符或遮盖型描述符"""
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)
        

class Managed: #5
    over = Overriding()
    over_no_get = OverridingNoGet()
    non_over = NonOverriding()
    
    def spam(self): #6
        print('-> Managed.spam({})'.format(display(self)))

In [2]:
obj = Managed()

In [3]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [4]:
Managed.over

-> Overriding.__get__(<Overriding object>, None, <class Managed>)


In [5]:
obj.over = 7

-> Overriding.__set__(<Overriding object>, <Managed object>, 7)


In [6]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [7]:
obj.__dict__['over'] = 8

In [8]:
vars(obj)

{'over': 8}

In [9]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [10]:
obj.over_no_get

In [11]:
Managed.over_no_get

In [12]:
obj.over_no_get = 7

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [13]:
obj.over_no_get

In [14]:
obj.__dict__['over_no_get'] = 9

In [15]:
obj.over_no_get

9

In [16]:
obj.over_no_get = 7

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [17]:
obj.over_no_get

9

In [18]:
obj = Managed()

In [19]:
obj.non_over

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)


In [20]:
obj.non_over = 7

In [21]:
obj.non_over

7

In [22]:
Managed.non_over

-> NonOverriding.__get__(<NonOverriding object>, None, <class Managed>)


In [23]:
del obj.non_over

In [24]:
obj.non_over

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)


In [25]:
obj = Managed()

In [26]:
Managed.over = 1

In [27]:
Managed.over_no_get = 2

In [28]:
Managed.non_over = 3

In [29]:
obj.over

1

In [30]:
obj.over_no_get

2

In [31]:
obj.non_over

3

In [32]:
obj = Managed()

In [33]:
obj.spam

<bound method Managed.spam of <__main__.Managed object at 0x0000026DB1AE6248>>

In [34]:
Managed.spam

<function __main__.Managed.spam(self)>

In [35]:
obj.spam = 7

In [36]:
obj.spam

7

In [38]:
import collections

class Text(collections.UserString):
    
    def __repr__(self):
        return 'Text({!r})'.format(self.data)
    
    def reverse(self):
        return self[::-1]

In [39]:
word = Text('forward')

In [40]:
word

Text('forward')

In [41]:
word.reverse()

Text('drawrof')

In [42]:
type(Text.reverse), type(word.reverse)

(function, method)

In [43]:
list(map(Text.reverse, ['repaid', (10, 20, 30), Text('stressed')]))

['diaper', (30, 20, 10), Text('desserts')]

In [45]:
Text.reverse.__get__(word)

<bound method Text.reverse of Text('forward')>

In [46]:
Text.reverse.__get__(None, Text)

<function __main__.Text.reverse(self)>

In [47]:
word.reverse

<bound method Text.reverse of Text('forward')>

In [48]:
word.reverse.__self__

Text('forward')

In [49]:
word.reverse.__func__ is Text.reverse

True